In [1]:
import requests,bs4
import pandas as pd
import numpy as np

In [2]:
class Scimago_Extract:
    def __init__ (self):
        pass
    def _sl_name_path_find (self,tr_tag): 
        self.prefix = 'https://www.scimagojr.com/'
        self.sl = self.tr_tag.select('td')[0].getText()
        self.name = self.tr_tag.select('td')[1].getText()
        self.sc_path = self.prefix + self.tr_tag.select('td[class="tit"]')[0].select('a')[0].get('href')
    def _scop_find(self,sc_path): 
        self.sc_html = requests.get(self.sc_path)
        self.sc_soup = bs4.BeautifulSoup(self.sc_html.text)
        self.scope = self.sc_soup.select('div[class="fullwidth"]')[0].contents[2]
        self.scope = self.scope.replace("\r","")
        self.scope = self.scope.replace("\n","") 
    def data_extract_single_page(self,soup): 
        self.soup = soup
        self.arr = []
        for self.tr_tag in self.soup.select('tr'):
            try:
                self._sl_name_path_find(self.tr_tag)
                self._scop_find(self.sc_path)
                self.arr.append([self.sl,self.name,self.scope])
            except IndexError as error: 
                pass
        self.np_arr = np.array(self.arr)
        return self.np_arr 
    def stop_key (self,soup): 
        self.key =  self.soup.select('a[class=""]')[1].select('img')[0].attrs.get('title')
        self.flag_sk = 1
        self.n_path = ""
        if self.key=='next':
            self.n_path = self.prefix+ self.soup.select('a[class=""]')[1].attrs.get('href')
        elif self.key=='previous':
            self.flag_sk=0
            self.n_path = ""
        return self.flag_sk,self.n_path 

In [3]:
def main (): 
    path = 'https://www.scimagojr.com/journalrank.php?category=2205&type=j'
    csv_file_name = 'Civil Engineering Journal Scope'
    soup = bs4.BeautifulSoup(requests.get(path).text)
    extract = Scimago_Extract()
    flag,count = 1,1
    while flag == 1:
        np_sig = extract.data_extract_single_page(soup)
        if count==1:
            np_cum = np_sig
        else : 
            np_cum = np.append(np_cum,np_sig,axis=0)
            
        flag, path = extract.stop_key(soup)
        if not path=="":  
            soup = bs4.BeautifulSoup(requests.get(path).text)
        count +=1
    pd.DataFrame(np_cum, columns=['SL','Name','Scope']).to_csv(csv_file_name+'.csv',index=False)
    print("Done")

In [4]:
main()

Done
